In [2]:
import pandas as pd
df = pd.read_csv('https://cghub.ucsc.edu/reports/SUMMARY_STATS/LATEST_MANIFEST.tsv',
                 low_memory=False, sep='\t')

In [3]:
# Last time it's run:
import datetime
print(datetime.datetime.today().date())

2016-03-24


In [4]:
# only interested in those with state equals Live
ndf = df[(df.study == 'CCLE') & (df.library_type == 'RNA-Seq') & (df.state == 'Live')]

In [5]:
def count(df, colname):
    # it can be any column other than 'study'
    res = df[['study', colname]].groupby(colname).count().sort_values('study')
    res['percent'] = (res.study / res.sum().values[0]).apply('{0:.2%}'.format)
    return res

## Library types

In [6]:
count(ndf, 'library_type')

,study,percent
library_type,,
RNA-Seq,935,100.00%


## File types

In [7]:
count(ndf, 'file_type')

,study,percent
file_type,,
bam,935,100.00%


## Number of samples

In [8]:
ndf.shape[0]

935

## File sizes

In [9]:
# Note: the count doesn't make sense when converting to filesize, so it's shown separately
from misc.utils import sizeof_fmt
# args: pass a sep character for better readbility. See sizeof_fmt?? for more details
ndf.files_size.describe().apply(sizeof_fmt, args=(' ',))

count    935.0 Bytes
mean         13.8 GB
std           3.3 GB
min           4.8 GB
25%          11.7 GB
50%          13.4 GB
75%          16.1 GB
max          34.2 GB
Name: files_size, dtype: object

#### Would be interesting to find out what the smallest and biggest files are

In [10]:
cols_to_show = ['disease_name', 'sample_type_name', 'filename', 'file_type',
                'files_size', 'files_size_readable', 'library_type', 'state']
# turn off warning, http://stackoverflow.com/questions/20625582/how-to-deal-with-this-pandas-warning
ndf.is_copy = False
ndf['files_size_readable'] = ndf.files_size.apply(sizeof_fmt)
sorted_ndf = ndf.sort_values('files_size').reset_index()
sorted_ndf.ix[[0,sorted_ndf.shape[0]-1]][cols_to_show]

,disease_name,sample_type_name,filename,file_type,files_size,files_size_readable,library_type,state
0,BREAST INVASIVE CARCINOMA,Cell Lines,G28034.MDA-MB-361.1.bam,bam,5.159984e+09,4.8 GB,RNA-Seq,Live
934,UTERINE CORPUS ENDOMETRIOID CARCINOMA,Cell Lines,G41715.JHUEM-1.5.bam,bam,3.676008e+10,34.2 GB,RNA-Seq,Live
